In [32]:
import pandas as pd
from sqlalchemy import create_engine
pd.set_option('display.max_columns', None)

import os


username = "root"
password = "Totodile561!"  
host = 'localhost'  
port = '3306'       
database = 'football_db'

engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}")

tables = engine.table_names()

player_tables = [table for table in tables if "player" in table]
dfs = {}

for table in player_tables:
    df = pd.read_sql_table(table, con=engine)
    dfs[table] = df
    print(f"Loaded DataFrame for table: {table}")

C:\Users\james\AppData\Local\Temp\ipykernel_15504\2692661351.py:16: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  tables = engine.table_names()


Loaded DataFrame for table: raw_player_defense_df
Loaded DataFrame for table: raw_player_gca_df
Loaded DataFrame for table: raw_player_keeper_df
Loaded DataFrame for table: raw_player_keepersadv_df
Loaded DataFrame for table: raw_player_passing_df
Loaded DataFrame for table: raw_player_passing_type_df
Loaded DataFrame for table: raw_player_possession_df
Loaded DataFrame for table: raw_player_shooting_df
Loaded DataFrame for table: raw_player_standard_stats_df


In [33]:
dfs.keys()
#dfs["player_defense_df"].head()

dict_keys(['raw_player_defense_df', 'raw_player_gca_df', 'raw_player_keeper_df', 'raw_player_keepersadv_df', 'raw_player_passing_df', 'raw_player_passing_type_df', 'raw_player_possession_df', 'raw_player_shooting_df', 'raw_player_standard_stats_df'])

In [34]:
import hashlib

def make_id(row):
    unique_str = f"{row['Player']}_{row['Nation']}_{row['Born']}"
    return hashlib.md5(unique_str.encode()).hexdigest()

for name, df in dfs.items():
    df['id'] = df.apply(make_id, axis=1)
    dfs[name] = df

In [ ]:
drop_rules = {
    'eredivisie': list(range(2010, 2018)),
    'championship': list(range(2010, 2018)),
    'primeira liga': list(range(2010, 2018)),
    'bundesliga': list(range(2010, 2017)),
    'premier league': list(range(2010, 2017)),
    'la liga': list(range(2010, 2017)),
    'serie a': list(range(2010, 2017)),
    'ligue 1': list(range(2010, 2017)),
    'scottish premiership': list(range(2010, 2026))
}

exclude_tables = {
    "raw_stats_player_standard_df",  
    "raw_player_keeper_df",
    "raw_player_shooting_df",   
}
import pandas as pd

def clean_text_columns(df, cols):
    for col in cols:
        if col in df.columns:
            df[col] = df[col].astype(str).str.strip().str.lower()
    return df

def extract_league_and_season(df):
    if 'League' not in df.columns:
        return df

    df[['league_name', 'season']] = df['League'].str.rsplit(' ', n=1, expand=True)
    df['season_start'] = pd.to_numeric(df['season'].str[:4], errors='coerce')
    df = df.dropna(subset=['season_start'])
    df['season_start'] = df['season_start'].astype(int)
    return df

def auto_int_convert(df):
    for col in df.columns:
        if pd.api.types.is_float_dtype(df[col]):
            if df[col].dropna().apply(float.is_integer).all():
                df[col] = df[col].astype('Int64')  
    return df

def apply_drop_rules(df, drop_rules):
    if 'league_name' not in df.columns or 'season_start' not in df.columns:
        return df

    mask = df.apply(
        lambda row: row['season_start'] in drop_rules.get(row['league_name'], []),
        axis=1
    )
    return df[~mask]

def process_player_df(df, drop_rules,skip_df=False):
    df = auto_int_convert(df)
    df = extract_league_and_season(df)
    df = clean_text_columns(df, ['Player', 'Nation', 'Squad', 'League','league_name','Pos'])

    df = df.dropna(subset=['Player', 'Nation', 'Born'], how='all')
    #df = df[df['Player'].str.lower() != 'player']

    if not skip_df:
        df = apply_drop_rules(df, drop_rules)
    return df

for key, df in dfs.items():
    if isinstance(df, pd.DataFrame):
        skip = key in exclude_tables
        dfs[key] = process_player_df(df, drop_rules,skip_df=skip)


In [40]:
dfs["raw_player_defense_df"]
#dfs["raw_player_possession_df"].dtypes



,Rk,Player,Nation,Pos,Squad,Age,Born,90s,Tkl,TklW,Def 3rd,Mid 3rd,Att 3rd,Tkl.1,Att,Tkl%,Lost,Blocks,Sh,Pass,Int,Tkl+Int,Clr,Err,Matches,League,id,league_name,season,season_start
0,1,junior adamu,at aut,fw,freiburg,23,2001,17.2,9,3,3,3,3,3,11,27.3,8,14,0,14,12,21,20,0,Matches,bundesliga 2024-2025,eaf12942ad4cd97ae15df30747b2b5c9,bundesliga,2024-2025,2024
1,2,karim adeyemi,de ger,"fw,mf",dortmund,22,2002,15.9,20,14,6,6,8,8,20,40.0,12,14,1,13,7,27,5,0,Matches,bundesliga 2024-2025,9352d43c4266a2f3df4c9daf87b7fc24,bundesliga,2024-2025,2024
2,3,amine adli,ma mar,"mf,fw",leverkusen,24,2000,8.5,15,9,4,2,9,4,11,36.4,7,11,0,11,0,15,4,0,Matches,bundesliga 2024-2025,37422c54b8710363f1b06d572170876c,bundesliga,2024-2025,2024
3,4,oladapo afolayan,eng eng,"fw,mf",st. pauli,26,1998,18.2,17,8,4,8,5,7,19,36.8,12,17,1,16,8,25,20,0,Matches,bundesliga 2024-2025,878ec37d57e37eb97e01fd5066045367,bundesliga,2024-2025,2024
4,5,felix agu,ng nga,df,werder bremen,24,1999,19.5,30,14,12,14,4,12,23,52.2,11,27,9,18,21,51,48,0,Matches,bundesliga 2024-2025,23a598dab48f93c3025bc61c7e10a34c,bundesliga,2024-2025,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37707,549,andre-frank zambo anguissa,cm cmr,mf,marseille,21,1995,27.1,88,65,23,57,8,32,94,34.0,62,50,7,43,67,155,46,3,Matches,ligue 1 2017-2018,fc5ab3a34497db3bb3ed9141c7a4989a,ligue 1,2017-2018,2017
37708,550,mehdi zeffane,dz alg,"df,mf",rennes,25,1992,11.0,17,12,10,5,2,9,25,36.0,16,10,2,8,12,29,44,0,Matches,ligue 1 2017-2018,a3772ae9fed1f8e0900ef72cab888fc0,ligue 1,2017-2018,2017
37709,551,erwin zelazny,fr fra,gk,troyes,25,1991,17.0,1,1,1,0,0,1,2,50.0,1,0,0,0,2,3,10,1,Matches,ligue 1 2017-2018,f03a88cc16820347e0630def00beba56,ligue 1,2017-2018,2017
37710,552,kévin zohi,ml mli,mf,strasbourg,20,1996,0.0,0,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,Matches,ligue 1 2017-2018,3eea21b8fd1f72970b941c86a52d4f90,ligue 1,2017-2018,2017


In [42]:

# Count nulls per column
dfs["raw_player_standard_stats_df"].isna().sum()

# Or check percentage of missing values
dfs["raw_player_standard_stats_df"].isna().mean() * 100


Rk              0.000000
Player          0.000000
Nation          0.000000
Pos             0.000000
Squad           0.000000
Age             0.099695
Born            0.099695
MP              0.000000
Starts          0.000000
Min             0.002848
90s             0.000000
Gls             0.002848
Ast             0.002848
G+A             0.002848
G-PK            0.002848
PK              0.002848
PKatt           0.002848
CrdY            0.000000
CrdR            0.000000
xG              0.011394
npxG            0.011394
xAG             0.011394
npxG+xAG        0.011394
PrgC            0.011394
PrgP            0.011394
PrgR            0.011394
Gls.1           0.002848
Ast.1           0.002848
G+A.1           0.002848
G-PK.1          0.002848
G+A-PK          0.002848
xG.1            0.014242
xAG.1           0.014242
xG+xAG          0.014242
npxG.1          0.014242
npxG+xAG.1      0.014242
Matches         0.000000
League          0.000000
id              0.000000
league_name     0.000000
